In [1]:
import os
import collections
import re
import json
import spacy
import pandas as pd
data_dir = "/home/stavros/DATA/TripAdvisorReviews/kresten_palace_hotel_&_wellness'"

In [2]:
data = pd.read_csv(os.path.join(data_dir, "kresten_palace_hotel_&_wellness'_365reviews.csv"))
print(data.shape)
print(data.columns)

published_year = data.publishedDate.apply(lambda x: x.split("-")[0])
published_year.value_counts()

(365, 24)
Index(['id', 'absoluteUrl', 'createdDate', 'publishedDate', 'locationId',
       'originalLanguage', 'language', 'stayDate', 'tripType', 'helpfulVotes',
       'title', 'text', 'rating', 'additionalRatings', 'username', 'userId',
       'user_hometownId', 'user_hometownName', 'response_id',
       'response_publishedDate', 'response_language', 'response_username',
       'response_connectionToSubject', 'response_text'],
      dtype='object')


2019    86
2016    48
2018    47
2017    45
2015    41
2014    29
2012    20
2011    14
2013    13
2010     8
2005     5
2006     3
2007     2
2003     1
2009     1
2004     1
2008     1
Name: publishedDate, dtype: int64

Expand contractions

In [3]:
_CMAP_DIR = os.path.join("/home/stavros/GitHub/Review-Aspects-App/nlptools", "contractions.txt")
with open(_CMAP_DIR, "r") as file:
  _CMAP = json.load(file)


def expand_contractions(text, contraction_mapping=_CMAP):
  contractions_pattern = re.compile('({})'.format('|'.join(
      contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)

  def expand_match(contraction):
    match = contraction.group(0)
    first_char = match[0]
    if contraction_mapping.get(match):
      expanded_contraction = contraction_mapping.get(match)
    else:
      expanded_contraction = contraction_mapping.get(match.lower())
    expanded_contraction = first_char+expanded_contraction[1:]
    return expanded_contraction

  expanded_text = contractions_pattern.sub(expand_match, text)
  expanded_text = re.sub("'s", "", expanded_text)
  expanded_text = re.sub("'", "", expanded_text)
  return expanded_text

In [4]:
expanded_text = data.text.apply(lambda x: expand_contractions(x))

Define stop word and special character sets

In [6]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at
back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by
call can cannot ca could
did do does doing done down due during
each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except
few fifteen fifty first five for former formerly forty four from front full
further
get give go
had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred
i if in indeed into is it its itself
keep
last latter latterly least less
just
made make many may me meanwhile might mine more moreover most mostly move much
must my myself
name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere
of off often on once one only onto or other others otherwise our ours ourselves
out over own
part per perhaps please put
quite
rather re really regarding
same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such
take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two
under until up unless upon us used using
various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would
yet you your yours yourself yourselves
""".split()
)
len(STOP_WORDS)

305

In [8]:
SPECIAL_CHARS = set('.,!?/\n;" ()-_&:')
for n in range(5): SPECIAL_CHARS.add(n * '\n')
for n in range(5): SPECIAL_CHARS.add(n * ' ')
SPECIAL_CHARS.add("...")
print(SPECIAL_CHARS)

INVALID_TOKENS = STOP_WORDS | SPECIAL_CHARS

{'', ' ', ':', ')', '&', '\n\n\n', '(', '!', ',', '    ', '-', ';', '  ', '...', '.', '"', '   ', '\n', '\n\n\n\n', '\n\n', '_', '?', '/'}


Apply spacy

In [5]:
nlp = spacy.load('en_core_web_sm')
processed_text = list(nlp.pipe(expanded_text))

Find most common words

In [9]:
text_to_token = {} # Dict[str, List[spacy.Token]]
for review in processed_text:
    for token in review:
        text = token.text.lower()
        if text not in INVALID_TOKENS:
            if text in text_to_token:
                text_to_token[text].append(token)
            else:
                text_to_token[text] = [token]
                
text_score = collections.Counter({text: len(tokens) for text, tokens in text_to_token.items()})

In [10]:
text_score.most_common()

[('hotel', 1059),
 ('room', 542),
 ('pool', 512),
 ('staff', 453),
 ('good', 430),
 ('food', 416),
 ('great', 290),
 ('nice', 264),
 ('day', 258),
 ('bar', 251),
 ('rhodes', 238),
 ('beach', 222),
 ('friendly', 208),
 ('rooms', 206),
 ('clean', 205),
 ('lovely', 205),
 ('restaurant', 201),
 ('night', 196),
 ('town', 168),
 ('kresten', 168),
 ('time', 167),
 ('area', 167),
 ('evening', 158),
 ('bus', 157),
 ('reception', 155),
 ('water', 152),
 ('like', 147),
 ('drinks', 142),
 ('2', 141),
 ('palace', 141),
 ('view', 136),
 ('greek', 132),
 ('stay', 131),
 ('breakfast', 127),
 ('4', 124),
 ('holiday', 124),
 ('entertainment', 123),
 ('sea', 122),
 ('week', 122),
 ('dinner', 117),
 ('air', 114),
 ('service', 114),
 ('little', 111),
 ('beds', 106),
 ('family', 106),
 ('walk', 106),
 ('people', 105),
 ('€', 105),
 ('plenty', 104),
 ('recommend', 104),
 ('small', 103),
 ('went', 102),
 ('kids', 102),
 ('stayed', 100),
 ('star', 97),
 ('bit', 96),
 ('got', 94),
 ('lots', 94),
 ('taxi', 93),
